In [2]:
import pandas as pd
pd.set_option('display.max_columns', None)
import os

In [3]:
script_directory = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
relative_folder_path = r"data\rangos_normales"
folder_path = os.path.join(script_directory.replace('\code', ''), relative_folder_path)
file = os.listdir(folder_path)
path = f"{folder_path}\{file[0]}"
df_root = pd.read_csv(path, sep='\t')

,field,min,max,gender
0,"Aspartate Aminotransferase (AST), Result (U/L)",11,34,0
1,"Alanine Aminotransferase (ALT), Result (U/L)",0,45,1
2,"Alanine Aminotransferase (ALT), Result (U/L)",0,34,2
3,"Serum Creatinine, Result (mg/dL)","0,73","1,18",1
4,"Serum Creatinine, Result (mg/dL)","0,55","1,02",2
5,"Creatine Kinase (CK), Result (U/L)",30,200,1
6,"Creatine Kinase (CK), Result (U/L)",29,168,2


In [15]:
for i in df_root['field'].tolist():
    print(f'"{i}"')

"Aspartate Aminotransferase (AST), Result (U/L)"
"Alanine Aminotransferase (ALT), Result (U/L)"
"Alanine Aminotransferase (ALT), Result (U/L)"
"Serum Creatinine, Result (mg/dL)"
"Serum Creatinine, Result (mg/dL)"
"Creatine Kinase (CK), Result (U/L)"
"Creatine Kinase (CK), Result (U/L)"


# Rangos normales code

### Accesing when gender is required

In [18]:

# Normals ranges file
script_directory = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
relative_folder_path = r"data\rangos_normales"
folder_path = os.path.join(script_directory.replace('\code', ''), relative_folder_path)
file = os.listdir(folder_path)
path = f"{folder_path}\{[x for x in file if 'chemestry_D1' in x][0]}" 
df_normal_ranges = pd.read_csv(path, sep=';')

In [4]:
float(df_normal_ranges[(df_normal_ranges['field']== "Alanine Aminotransferase (ALT), Result (U/L)") & (df_normal_ranges['gender']== 1)]['min'].iloc[0])

0.0

In [19]:
float(df_normal_ranges[(df_normal_ranges['field']== "Serum Creatinine, Result (mg/dL)") & (df_normal_ranges['gender']== 2)]['min'].iloc[0])

0.55

In [28]:
df_normal_ranges.head()

,field,min,max,gender,age_min,age_max
0,"Aspartate Aminotransferase (AST), Result (U/L)",11,34,0,NaN,NaN
1,"Alanine Aminotransferase (ALT), Result (U/L)",0,45,1,NaN,NaN
2,"Alanine Aminotransferase (ALT), Result (U/L)",0,34,2,NaN,NaN
3,"Serum Creatinine, Result (mg/dL)","0,73","1,18",1,NaN,NaN
4,"Serum Creatinine, Result (mg/dL)","0,55","1,02",2,NaN,NaN


In [5]:
df_normal_ranges[(df_normal_ranges['field']=="Aspartate Aminotransferase (AST), Result (U/L)") & (df_normal_ranges['gender']== 0)]['min'].iloc[0]
#df_root[(df_root['field'] == "Aspartate Aminotransferase (AST), Result (U/L)") & (df_root['gender'] == 1)]['min'].iloc[0]


'11'

### Accesing when no condition of gender or age

In [31]:
df_normal_ranges[df_normal_ranges['field']=="Aspartate Aminotransferase (AST), Result (U/L)"]['min'].iloc[0]

'11'

In [13]:
print(float(min))

11.0


In [3]:
from datetime import datetime

print(float((datetime.strptime('11:00', '%H:%M') - datetime.strptime('10:00', '%H:%M')).total_seconds() / 60))

60.0


In [3]:
for i in df_root.Campo.unique():
    if 'rand' in i.lower():
        print(i)

Is the participant eligible to randomization?
Will the participant be randomized?
Is the subject replacing a subject previously been randomized in the current study?


In [16]:
df_root.Visit.unique() 

array(['Screening Visit', 'D-1', 'D1', 'D2', 'D3', 'D4', 'D7', 'D14',
       'D15', 'D16', 'D17', 'D18', 'D21', 'D28', 'D29', 'D30', 'D31',
       'D32', 'D35', 'D42', 'D63', '-'], dtype=object)

In [27]:
df_root.columns

Index(['Id Form', 'idCampo', 'Site', 'Visit', 'name', 'Genero',
       'AñoNacimiento', 'NameActividad', 'activityState', 'fecha enrolado',
       'Participante', 'Estado del Participante', 'identifierPrefix',
       'Id Activityins', 'Instancia', 'Estado del campo', 'Campo', 'Valor',
       'displayName', 'id option', 'Variable', 'visible', 'required',
       'TipoDato', 'TypeQuery', 'MessageQuery', 'QueryState',
       'vistaPrincipal.IdSubject'],
      dtype='object')

In [9]:
df_root.name.unique()

array(['Date of visit', 'Informed Consent', 'Demographics',
       'History of cutaneous leishmaniasis', 'Covid 19 testing',
       'Eligibility',
       'Medical Or Surgical History (other than Leishmaniasis)',
       'Vein assessment', 'Urinary Drug Screen', 'Alcohol Screen',
       'Clinical Laboratory - Test Hematology',
       'Clinical Laboratory Test - Clinical Chemistry',
       'Clinical Laboratory Test - Coagulation',
       'Immunoassay (Thyroid Stimulating Hormone)', 'Urinalysis',
       'Urine Microscopic Examination', 'Virology', 'Lesion Measurement',
       'Physical Examination', 'Vital Signs', '12-Lead ECG',
       'Clinical Laboratory Test - Clinical Chemistry D-1',
       'Interleukin-6', 'Titration Of Auto-Antibodies',
       'End of Study Treatment (Miltefosine)',
       'Injection Site Examination', 'PBMC Isolate',
       'Pharmacokinetic Blood Sampling (PK)',
       'Pharmacodynamic Blood Sampling (PD) -Cytokines/Chemokines',
       'mRNA Markers', 'CpG ODN D35 A

In [5]:
df_root.Visit.unique()  

array(['Screening Visit', 'D-1', 'D1', 'D2', 'D3', 'D4', 'D7', 'D14',
       'D15', 'D16', 'D17', 'D18', 'D21', 'D28', 'D29', 'D30', 'D31',
       'D32', 'D35', 'D42', 'D63', '-'], dtype=object)

In [5]:
formulario_elegido =   'Medical Or Surgical History (other than Leishmaniasis)'
visita_elegida = 0
campo_elegido = 0

general_revision  = df_root[df_root['name']== formulario_elegido]
general_revision = general_revision[['Visit','Participante', 'Campo', 'Valor']]
if visita_elegida != 0:
    general_revision = general_revision[general_revision['Visit'] == visita_elegida]

if campo_elegido != 0:
    general_revision = general_revision[general_revision['Campo']== campo_elegido]

general_revision = general_revision[['Visit', 'Campo', 'Participante','Valor']]

In [8]:
general_revision[general_revision['Participante']=='011002']

,Visit,Campo,Participante,Valor
3205,Screening Visit,Are there any relevant medical history or surg...,011002,1
3621,Screening Visit,Medical/Surgical History/Current Condition,011002,tubectomy
3622,Screening Visit,Onset Date/First Diagnosis/Surgery,011002,22-JUN-2023
3623,Screening Visit,Is Condition Ongoing?,011002,0
3624,Screening Visit,End Date,011002,22-JUN-2023
3625,Screening Visit,Medical/Surgical History/Current Condition,011002,unspecified acne
3626,Screening Visit,Onset Date/First Diagnosis/Surgery,011002,UNK-DEC-2023
3627,Screening Visit,Is Condition Ongoing?,011002,1
3628,Screening Visit,Severity,011002,1
3629,Screening Visit,Frequency,011002,2


In [9]:
for i in general_revision.Campo.unique():
    print(f"'{i}'")

'End of study/Early withdrawal date'
'Did the participant complete the study?'
'Date of last study treatment administration'
'Was the study treatment completed per protocol?'
'Primary reason for not completing the study'


In [74]:
general_revision.head(70)

,Visit,Campo,Participante,Valor
302,-,End of study/Early withdrawal date,011001,26-JAN-2024
303,-,Did the participant complete the study?,011001,1
304,-,Date of last study treatment administration,011001,28-NOV-2023
305,-,Was the study treatment completed per protocol?,011001,1
2695,-,End of study/Early withdrawal date,S002,27-NOV-2023
2696,-,Did the participant complete the study?,S002,0
2697,-,Primary reason for not completing the study,S002,1
2715,-,End of study/Early withdrawal date,S003,08-NOV-2023
2716,-,Did the participant complete the study?,S003,0
2717,-,Primary reason for not completing the study,S003,1


In [27]:
general_revision[general_revision['Participante']== '011001' ].head(50)

,Visit,Campo,Participante,Valor
302,-,End of study/Early withdrawal date,011001,26-JAN-2024
303,-,Did the participant complete the study?,011001,1
304,-,Date of last study treatment administration,011001,28-NOV-2023
305,-,Was the study treatment completed per protocol?,011001,1


In [30]:
general_revision[general_revision['Campo']== 'Date of dosing' ].head(50)

,Visit,Campo,Participante,Valor
2603,-,Date of dosing,011001,31-OCT-2023
2888,-,Date of dosing,011001,14-NOV-2023
2894,-,Date of dosing,011001,28-NOV-2023


In [10]:
formulario_elegido =  'Prior And Concomitant Medications'
visita_elegida = 0
campo_elegido = 0

general_revision  = df_root[df_root['name']== formulario_elegido]
general_revision = general_revision[['Visit','Participante', 'Campo', 'Valor']]
if visita_elegida != 0:
    general_revision = general_revision[general_revision['Visit']in ['D2', 'D16', 'D30']]

if campo_elegido != 0:
    general_revision = general_revision[general_revision['Campo']== campo_elegido]

general_revision = general_revision[['Visit', 'Campo', 'Participante','Valor']]

In [14]:
general_revision

,Visit,Campo,Participante,Valor
3610,-,Concomitant Medication ID,S005,1
3611,-,Drug Name,S005,omeprozole
3612,-,Indication,S005,chronic gastritis
3613,-,Indication Category,S005,2
3614,-,Medical History ID,S005,1
3615,-,Dose per Administration,S005,20
3616,-,Unit,S005,mg
3617,-,Frequency,S005,1
3618,-,Route,S005,1
3619,-,Start date,S005,UNK-UNK-2022


In [8]:
general_revision[general_revision['Campo']=='60-mins post dose, Time']

,Visit,Campo,Participante,Valor


In [9]:
df__lession  = df_root[df_root['name']=='Lesion Measurement']
df__lession = df__lession[['Visit','Participante', 'Campo', 'Valor']]
df__lession = df__lession[df__lession['Visit']=='Screening Visit']
df__lession = df__lession[df__lession['Campo']=='Anatomical Location']
df__lession = df__lession[['Visit','Participante','Valor']]
df__lession = df__lession.rename(columns={'Participante':'Subject', 'Valor':'lesion_measurement'})
df__lession['Count'] = len(df__lession['lesion_measurement'])
df__lession = df__lession[['Visit','Subject','Count']]
# df_with_counts = df__lession.merge(subject_counts, left_on='Subject', right_on='Subject')
# df_with_counts = df_with_counts[['Visit','Subject','Count']]

In [10]:
df_root[df_root['Campo']=='Date of ECG Performed'].Valor.unique()

array(['25-OCT-2023', '31-OCT-2023', '14-NOV-2023', '28-NOV-2023',
       '12-DEC-2023'], dtype=object)

In [ ]:
df__lession

,Visit,Subject,Count
300,Screening Visit,011001,2
309,Screening Visit,011001,2


In [ ]:
df_root.activityState.unique()

array(['DATA_VERIFIED', 'DATA_ENTRY_COMPLETE', 'DATA_ENTRY_INCOMPLETE',
       'DATA_EMPTY'], dtype=object)

In [ ]:
df_root.Participante.unique()

array(['011001', 'S002', 'S003'], dtype=object)

In [ ]:
prueba = df_root[df_root['name']=='CpG ODN D35 Administration'].sort_values(by='Instancia')
prueba = prueba[(prueba['Campo']=='Date of dosing') | (prueba['Campo']=='Time of Dosing')]
prueba1 = prueba[prueba['Campo']=='Date of dosing']
prueba1['time_dosing_cpg_administration'] =  prueba[prueba['Instancia'].isin(prueba1['Instancia'] + 1) & (prueba['Campo'] == 'Time of Dosing')]['Valor'].values
prueba1 =prueba1[['Participante', 'Campo', 'Valor', 'time_dosing_cpg_administration' ,'Instancia']]
prueba1 = prueba1.rename(columns={'Participante':'Subject', 'Valor':'date_cpg_administration'})
prueba1


C:\Users\sebastian sossa\AppData\Local\Temp\ipykernel_21716\1665216494.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prueba1['time_dosing_cpg_administration'] =  prueba[prueba['Instancia'].isin(prueba1['Instancia'] + 1) & (prueba['Campo'] == 'Time of Dosing')]['Valor'].values


,Subject,Campo,date_cpg_administration,time_dosing_cpg_administration,Instancia
2603,011001,Date of dosing,31-OCT-2023,09:36,17797
2888,011001,Date of dosing,14-NOV-2023,09:05,21664
2894,011001,Date of dosing,28-NOV-2023,10:12,21678


In [ ]:
df_time_dosing1_miltefosine_miltefosine_miltefosine_miltefosine = df_root[df_root['name']=='CpG ODN D35 Administration'].sort_values(by='Instancia')
df_time_milteosine = df_time_milteosine[(df_time_milteosine['Campo']=='Date of dosing') | (df_time_milteosine['Campo']=='Time of Dosing')]
df_time_dosing = df_time_milteosine[df_time_milteosine['Campo']=='Date of dosing']
df_time_dosing['time_dosing_cpg_administration'] =  df_time_milteosine[df_time_milteosine['Instancia'].isin(df_time_dosing['Instancia'] + 1) & (df_time_milteosine['Campo'] == 'Time of Dosing')]['Valor'].values
df_time_dosing =df_time_dosing[['Participante', 'Campo', 'Valor', 'time_dosing_cpg_administration']]
df_time_dosing = df_time_dosing.rename(columns={'Participante':'Subject', 'Valor':'date_ex_to_join'})

C:\Users\sebastian sossa\AppData\Local\Temp\ipykernel_21716\1862077006.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time_dosing['time_dosing_cpg_administration'] =  df_time_dosing1[df_time_dosing1['Instancia'].isin(df_time_dosing['Instancia'] + 1) & (df_time_dosing1['Campo'] == 'Time of Dosing')]['Valor'].values


In [ ]:
df_time_dosing

,Subject,Campo,date_ex_to_join,time_dosing_cpg_administration
2603,011001,Date of dosing,31-OCT-2023,09:36
2888,011001,Date of dosing,14-NOV-2023,09:05
2894,011001,Date of dosing,28-NOV-2023,10:12


In [ ]:
df_time_dosing = df_root[df_root['name']=='CpG ODN D35 Administration']
df_time_dosing = df_time_dosing[['Participante', 'Campo', 'Valor', 'FormFieldInstance Id']]
df_time_dosing = df_time_dosing[df_time_dosing['Campo']=='Was the visit performed?']
df_time_dosing = df_time_dosing[['Visit','Participante','Valor_completo']]
df_time_dosing = df_time_dosing.rename(columns={'Participante':'Subject', 'Valor_completo':'was_DV_performed'})

KeyError: "['FormFieldInstance Id'] not in index"

In [18]:
df_contraception = df_root[df_root['name']== 'Prior And Concomitant Medications']
df_contraception = df_contraception[['Visit','Participante', 'Campo', 'Valor', 'Instancia']]
df_contraception = df_contraception.sort_values(by=['Instancia'])
df_contraception = df_contraception.reset_index(drop=True)
date_indices = df_contraception.index[df_contraception['Campo'] == 'Concomitant Medication ID'].tolist()

In [19]:
date_indices

[0]

In [16]:
df_contraception = df_root[df_root['name']== 'Prior And Concomitant Medications']
df_contraception = df_contraception[['Visit','Participante', 'Campo', 'Valor', 'Instancia']]
df_contraception = df_contraception.sort_values(by=['Instancia'])
df_contraception = df_contraception.reset_index(drop=True)
date_indices = df_contraception.index[df_contraception['Campo'] == 'Concomitant Medication ID'].tolist()
subdatasets = [df_contraception.iloc[start:end] for start, end in zip(date_indices, date_indices[1:] + [None])]
df_to_join = pd.DataFrame()
for sub in subdatasets:
    #if sub['Valor'].tolist()  in [5, 5.0, '5', '5.0', 9]:
    if len([float(x) for x in sub['Valor'].tolist() if x in [5, 5.0, '5', '5.0', 9]]) != 0:
        df_to_join = sub[(sub['Campo'] == 'Indication Category') | (sub['Campo'] == 'Start date')]
        sujeto = df_to_join.Participante.unique()
        df_to_join = df_to_join[['Campo', 'Valor']].T
        new_columns = df_to_join.iloc[0]
        df_to_join = df_to_join[1:].set_axis(new_columns, axis=1)
        df_to_join['Subject'] = sujeto
        break
df_to_join = df_to_join.rename(columns={'Participante':'Subject', 'Start date':'start_date_combined_hormonal'})

In [17]:
df_to_join

""


In [ ]:
pru.Campo.unique()

array(['Was the ECG performed?', 'Date of ECG Performed',
       'Undefined, HR (bpm)', 'Undefined, RR (msec)',
       'Undefined, PR (msec)', 'Undefined, QRS (msec)',
       'Undefined, QT (msec)', 'Undefined, QTcF (msec)',
       'Undefined, Interpretation', 'Pre dose triplicate 1, Time 24 hrs',
       'Pre dose triplicate 1, HR (bpm)',
       'Pre dose triplicate 1, RR (msec)',
       'Pre dose triplicate 1, PR (msec)',
       'Pre dose triplicate 1, QRS (msec)',
       'Pre dose triplicate 1, QT (msec)',
       'Pre dose triplicate 1, QTcF (msec)',
       'Pre dose triplicate 1, Interpretation',
       'Pre dose triplicate 1, If abnormal, specify',
       'Pre dose triplicate 1, Please specify abnormal interpretation',
       'Pre dose triplicate 2, Time 24 hrs',
       'Pre dose triplicate 2, HR (bpm)',
       'Pre dose triplicate 2, RR (msec)',
       'Pre dose triplicate 2, PR (msec)',
       'Pre dose triplicate 2, QRS (msec)',
       'Pre dose triplicate 2, QT (msec)',
       

In [ ]:
triplicate_1 = pru[pru['Campo']=='Pre dose triplicate 1, Time 24 hrs'].Valor.unique()[0]
triplicate_2 = pru[pru['Campo']=='Pre dose triplicate 2, Time 24 hrs'].Valor.unique()[0]
print(triplicate_1)
print(triplicate_2)

08:20
08:25


In [ ]:
from datetime import datetime
time_format = '%H:%M'
triplicate_1_formated = datetime.strptime(triplicate_1, time_format)
triplicate_2_formated = datetime.strptime(triplicate_2, time_format)

time_difference = (triplicate_2_formated - triplicate_1_formated).total_seconds() / 60
print(time_difference)


5.0


In [ ]:
import math
import numpy as np
prueba_variable = np.nan

if math.isnan(float(prueba_variable)):
    print('Si funciona asi')
else:
    print('No funciona asi') 

Si funciona asi


In [ ]:
import math

if math.nan > 4:
    print('Si toca cambiar')
else:
    print('no toca cambiar')

no toca cambiar


## Pruebas con datos root

In [3]:

# Normals ranges file
script_directory = os.path.dirname(os.path.abspath(__file__)) if '__file__' in locals() else os.getcwd()
relative_folder_path = "data"
folder_path = os.path.join(script_directory.replace('\code', ''), relative_folder_path)
file = os.listdir(folder_path)
path = f"{folder_path}\{file[0]}" 
df_root = pd.read_excel(path)

In [8]:
df_administration_miltefosine = df_root[df_root['name']==  'Miltefosine Administration']
df_administration_miltefosine = df_administration_miltefosine[['Visit','Participante', 'Campo', 'Valor']]
df_administration_miltefosine = df_administration_miltefosine[df_administration_miltefosine['Campo'] == 'Date of dosing']
df_administration_miltefosine['Valor'] = pd.to_datetime(df_administration_miltefosine['Valor'], format='%d-%b-%Y')
max_per_participante = df_administration_miltefosine.groupby('Participante')['Valor'].transform('min')
df_administration_miltefosine = df_administration_miltefosine[df_administration_miltefosine['Valor'] == max_per_participante]
df_administration_miltefosine['Valor'] = pd.to_datetime(df_administration_miltefosine['Valor']).dt.strftime('%d-%b-%Y').str.upper()
df_administration_miltefosine = df_administration_miltefosine[['Participante','Valor']].drop_duplicates()
df_administration_miltefosine = df_administration_miltefosine.rename(columns={'Participante':'Subject', 'Valor':'date_dosing_miltefosine'})

In [9]:
df_administration_miltefosine

,Subject,date_dosing_miltefosine
4660,011002,20-FEB-2024


In [14]:
df_adverse = df_root[df_root['name']=='Adverse Events']['Participante'].tolist()

In [16]:
set(df_adverse)

{'011002'}

In [10]:

df_adverse = df_root[df_root['name']=='Adverse Events']

In [49]:

dataframe_join_adverse = pd.DataFrame()
for sujeto_adverse in set(df_root[df_root['name']=='Adverse Events']['Participante'].tolist()):

    df_adverse = df_root[df_root['Participante']==sujeto_adverse]
    df_adverse = df_adverse[df_adverse['name']=='Adverse Events']

    df_adverse = df_adverse.sort_values(by=['Instancia'], ascending=True)
    df_adverse = df_adverse.reset_index(drop=True)

    date_indices = df_adverse.index[df_adverse['Campo'] == 'Adverse Event Reported Term'].tolist()
    subdatasets = [df_adverse.iloc[start:end] for start, end in zip(date_indices, date_indices[1:] + [None])]

    for subdataset in subdatasets:

        pru_adverse = subdataset[['Campo', 'Valor']].T
        new_columns_adverse = pru_adverse.iloc[0]
        pru_adverse = pru_adverse[1:].set_axis(new_columns_adverse, axis=1)
        pru_adverse['Subject'] = sujeto_adverse
        pru_adverse = pru_adverse[pru_adverse['Action taken with study treatment (Miltefosine)'].isin([1,2,3,4,5,6,7,8,9,99,'1','2','3','4','5','6','7','8','9','99',1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,99.0 ])]
        pru_adverse =pru_adverse[['Start Date', 'Action taken with study treatment (Miltefosine)']]
        pru_adverse = pru_adverse.rename(columns={'Start Date':'to_join', 'Action taken with study treatment (Miltefosine)': 'Action_taken_miltefosine' })
  
        dataframe_join_adverse = pd.concat([dataframe_join_adverse, pru_adverse], ignore_index=True)

        

In [50]:
dataframe_join_adverse

Campo,to_join,Action_taken_miltefosine
0,20-FEB-2024,1
1,21-FEB-2024,1
2,21-FEB-2024,1
3,01-MAR-2024,1
4,07-MAR-2024,1


In [25]:
df_concomitant_medication = df_root[df_root['name']== 'Prior And Concomitant Medications']
df_concomitant_medication = df_concomitant_medication[['Visit','Participante', 'Campo', 'Valor', 'Variable' ]]
df_concomitant_medication = df_concomitant_medication[df_concomitant_medication['Variable'].isin(['CMTRT'])]
df_concomitant_medication['concomitant_medication_name'] = df_concomitant_medication.groupby('Participante')['Valor'].transform('count')
df_concomitant_medication = df_concomitant_medication[['Participante', 'concomitant_medication_name']].drop_duplicates()
df_concomitant_medication = df_concomitant_medication.rename(columns={'Participante':'Subject'})

In [26]:
df_concomitant_medication

,Visit,Participante,Campo,Valor,Variable
3492,-,S005,Drug Name,omeprazole,CMTRT
4697,-,011002,Drug Name,omeprazole,CMTRT
4709,-,011002,Drug Name,Trimebutine,CMTRT
5007,-,011002,Drug Name,calendula suspension,CMTRT
5033,-,011002,Drug Name,omeprazole,CMTRT
5043,-,011002,Drug Name,calendula suspension,CMTRT
5053,-,011002,Drug Name,Bilastine,CMTRT
